In [1]:
import os
import numpy as np
env_var = os.environ
os.environ["LACE_REPO"] = "/nfs/pic.es/user/l/lcabayol/DESI/LaCE"
os.environ["LACE_MANAGER_REPO"] = "/nfs/pic.es/user/l/lcabayol/DESI/LaCE_manager"

In [2]:
from lace.emulator.nn_architecture import MDNemulator_polyfit
from lace.emulator.nn_emulator import NNEmulator
from lace.emulator.gp_emulator import GPEmulator

from lace.emulator.emulator import P1Demulator

from lace.emulator.test_simulation_nn import test_sim
from lace.emulator import pnd_archive

/data/astro/scratch/lcabayol/anaconda3/envs/DESIenv6/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# our modules
from lace.cosmo import fit_linP
from lace.emulator import poly_p1d


In [4]:
kmax_Mpc=4
ndeg=5
Nsim=30

emuparams = ['Delta2_p', 'n_p','mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']


# Neural Network

By default, the NNemulator is trained with the following parameters:ç
- zmax=4.5
- Nsim=30 
- nepochs=100
- step_size=75
- kmax_Mpc=4
- ndeg=5. For the extended version (kmax_Mpc=8), set to 7
- postprocessing='768'. To train with the 1D post-processing change to '500'

- initial_weights=True. Always starts from the same iniial parameters. These were selected randomly. Set to False if random initialization is required.

## Train the neural network from scratch from predefined weights and save it in a specified path

In [8]:
emulator = NNEmulator(emuparams,list_archives=['data_input_axes','data_input_phases'], ndeg=5, save_path=f'/nfs/pic.es/user/l/lcabayol/DESI/LaCE/lace/emulator/NNmodels/test.pt')






start the training of the emulator
Training network on 11550
Emualtor trained in 110.02759432792664 seconds


In [9]:
truth=pnd_archive.archivePND(z_max=4.5,nsamples=30, pick_sim='central')
truth.average_over_samples(flag="all")
zs = [d['z'] for d in truth.data_av_all if d['scale_tau'] == 1] 
truth = [d for d in truth.data_av_all if d['scale_tau'] == 1] 

fractional_error, emu_p1d, true_p1d, emu_p1derr = test_sim(emulator.emulator, truth, emulator.emuparams, emulator.paramLims, emulator.device, emulator.yscalings,  kmax_Mpc_test=kmax_Mpc, ndeg=ndeg )


42
Mean fractional error: 0.9976439590024025
Std fractional error: 0.004365362898353266


## Load a trained emulator

In [10]:
emulator = NNEmulator(emuparams, model_path=f'NNmodels/NNEmulator_LaCEHC.pt', train=False)




Model loaded. No training needed


In [11]:
truth=pnd_archive.archivePND(z_max=4.5,nsamples=30, pick_sim='central')
truth.average_over_samples(flag="all")
zs = [d['z'] for d in truth.data_av_all if d['scale_tau'] == 1] 
truth = [d for d in truth.data_av_all if d['scale_tau'] == 1] 

fractional_error, emu_p1d, true_p1d, emu_p1derr = test_sim(emulator.emulator, truth, emulator.emuparams, emulator.paramLims, emulator.device, emulator.yscalings,  kmax_Mpc_test=kmax_Mpc, ndeg=ndeg )


42
Mean fractional error: 0.999178485002622
Std fractional error: 0.004228317231333182


# Gaussian Process 

- It is trained on 330 P1D. By default, it does not use optical-depth rescalings
- The default options for $k$, $z$, and 'deg' are the same as for the NN emulator

In [12]:
emulator = GPEmulator()

 /data/astro/scratch/lcabayol/anaconda3/envs/DESIenv6/lib/python3.10/site-packages/paramz/parameterized.py:61: RuntimeWarning:Don't forget to initialize by self.initialize_parameter()!


Training GP on 330 points
GPs optimised in 0.43 seconds


In [13]:
truth=pnd_archive.archivePND(z_max=4.5, pick_sim='central')
truth.average_over_samples(flag="all")
truth = [d for d in truth.data_av_all if d['scale_tau'] == 1] 

for aa,item in enumerate(truth):
    # figure out redshift for this entry
    z=item["z"]

    true_k=item["k_Mpc"]
    k_mask=(true_k<kmax_Mpc) & (true_k>0)
    true_p1d=item["p1d_Mpc"][k_mask]
    coeff_p1d = poly_p1d.PolyP1D(true_k[k_mask],true_p1d,kmin_Mpc=1.e-3,kmax_Mpc=emulator.kmax_Mpc,deg=emulator.ndeg).lnP_fit
    poly=np.poly1d(coeff_p1d)
    true_p1d_poly =np.exp(poly(np.log(true_k[k_mask])))

    # true p1d (some sims have an extra k bin, so we need to define mask again)
    true_k=item["k_Mpc"]
    k_mask=(true_k<kmax_Mpc) & (true_k>0)
    true_p1d=item["p1d_Mpc"][k_mask]
    assert len(true_p1d)==(emulator.k_bin -1)


    # for each entry, figure emulator parameter describing it (labels)
    emu_call={}
    for bb,param in enumerate(emulator.paramList):
        emu_call[param]=item[param]

    # ask emulator to emulate P1D (and its uncertainty)
    emu_p1d,emu_err=emulator.emulate_p1d_Mpc(emu_call,emulator.training_k_bins,return_covar=True)
    fractional_errors_gp = emu_p1d/true_p1d_poly
print(f'mean fractional error: {np.mean(fractional_errors_gp)}')
print(f'std fractional error: {np.std(fractional_errors_gp)}')

mean fractional error: 0.9909920710299086
std fractional error: 0.004361883605622311



## Gaussian Process - version in Pedersen et al 2023

In [14]:
emulator = GPEmulator(postprocessing='500')

Training GP on 330 points
GPs optimised in 0.47 seconds


# MODULE COMBIING NN AND GP EMULATORS

## Default NN emulator training the emualtor from scrath and not saving it

In [15]:
P1Demulator(emu_algorithm='NN')

start the training of the emulator
Training network on 11550
Emualtor trained in 110.97567009925842 seconds


## Default GP emulator training the emualtor from scrath and not saving it

In [16]:
P1Demulator(emu_algorithm='GP', drop_rescalings=False)

Training GP on 330 points
GPs optimised in 0.46 seconds
